# Project 2.1

### Task 1

1. Сгенерировать DataFrame из трёх колонок (row_id, discipline, season) - олимпийские дисциплины по сезонам.  
•	row_id - число порядкового номера строки;  
•	discipline - наименование олимпиский дисциплины на английском (полностью маленькими буквами);  
•	season - сезон дисциплины (summer / winter);  
*Укажите не мнее чем по 5 дисциплин для каждого сезона.  
Сохраните DataFrame в csv-файл, разделитель колонок табуляция, первая строка должна содержать название колонок.  
Данные должны быть сохранены в виде 1 csv-файла а не множества маленьких.


In [1]:
import findspark
findspark.init('/opt/hadoop/spark')

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, broadcast

In [3]:
spark = (SparkSession
 .builder
 .appName('project_2_1')
 .enableHiveSupport()
 .getOrCreate())

/opt/hadoop/spark/python/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
schema = "row_id BIGINT, discipline STRING, season STRING"

In [5]:
rows = [
(1,"skateboarding", "summer"),(2,"figure skating", "winter"),
(3,"swimming", "summer"), (4,"ski jumping", "winter"),
(5,"surfing", "summer"), (6,"snowboard", "winter"),
(7,"golf", "summer"), (8,"ski mountaineering", "winter"),
(9,"athletics", "summer"), (10,"biathlon", "winter"),
(11,"badminton", "summer"), (12,"bobsleigh", "winter"),
(13,"boxing", "summer"), (14,"cross-country skiing", "winter"),
(15,"triathlon", "summer"), (16,"speed skating", "winter")
]

In [6]:
olympicDF = spark.createDataFrame(rows, schema)
olympicDF.show()

+------+--------------------+------+
|row_id|          discipline|season|
+------+--------------------+------+
|     1|       skateboarding|summer|
|     2|      figure skating|winter|
|     3|            swimming|summer|
|     4|         ski jumping|winter|
|     5|             surfing|summer|
|     6|           snowboard|winter|
|     7|                golf|summer|
|     8|  ski mountaineering|winter|
|     9|           athletics|summer|
|    10|            biathlon|winter|
|    11|           badminton|summer|
|    12|           bobsleigh|winter|
|    13|              boxing|summer|
|    14|cross-country skiing|winter|
|    15|           triathlon|summer|
|    16|       speed skating|winter|
+------+--------------------+------+



In [9]:
olympicDF.repartition(1).write.mode("overwrite").csv('file:///opt/notebooks/olympic_sports.csv', header=True, sep='\t')

### Task 2

2. Прочитайте исходный файл "Athletes.csv".  

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.  
Результат сохраните в формате parquet.  


In [10]:
athletesDF = spark.read.csv('file:///opt/notebooks/Athletes.csv', header=True, sep=';')

In [13]:
athletesDF.show(truncate=False)

+-----------------------+------------------------+-------------------+
|Name                   |NOC                     |Discipline         |
+-----------------------+------------------------+-------------------+
|AALERUD Katrine        |Norway                  |Cycling Road       |
|ABAD Nestor            |Spain                   |Artistic Gymnastics|
|ABAGNALE Giovanni      |Italy                   |Rowing             |
|ABALDE Alberto         |Spain                   |Basketball         |
|ABALDE Tamara          |Spain                   |Basketball         |
|ABALO Luc              |France                  |Handball           |
|ABAROA Cesar           |Chile                   |Rowing             |
|ABASS Abobakr          |Sudan                   |Swimming           |
|ABBASALI Hamideh       |Islamic Republic of Iran|Karate             |
|ABBASOV Islam          |Azerbaijan              |Wrestling          |
|ABBINGH Lois           |Netherlands             |Handball           |
|ABBOT

In [14]:
athletes_countDF = athletesDF.groupBy("Discipline").count()
athletes_countDF.show(10, truncate=False)

+---------------------+-----+
|Discipline           |count|
+---------------------+-----+
|Tennis               |178  |
|Boxing               |270  |
|Marathon Swimming    |49   |
|Golf                 |115  |
|Rowing               |496  |
|Baseball/Softball    |220  |
|Judo                 |373  |
|Sailing              |336  |
|Swimming             |743  |
|Cycling BMX Freestyle|19   |
+---------------------+-----+
only showing top 10 rows



In [15]:
athletes_countDF.count()

47

In [16]:
athletes_countDF.write.mode("overwrite").parquet('file:///opt/notebooks/athletes_count.parquet')

In [17]:
testDF = spark.read.parquet('file:///opt/notebooks/athletes_count.parquet')
testDF.show(100, truncate=False)

+---------------------+-----+
|Discipline           |count|
+---------------------+-----+
|Tennis               |178  |
|Boxing               |270  |
|Marathon Swimming    |49   |
|Golf                 |115  |
|Rowing               |496  |
|Baseball/Softball    |220  |
|Judo                 |373  |
|Sailing              |336  |
|Swimming             |743  |
|Cycling BMX Freestyle|19   |
|Basketball           |280  |
|Handball             |343  |
|Rhythmic Gymnastics  |95   |
|null                 |1    |
|Karate               |77   |
|Triathlon            |106  |
|Badminton            |164  |
|Canoe Sprint         |236  |
|Athletics            |2068 |
|Cycling Track        |208  |
|Beach Volleyball     |90   |
|Skateboarding        |77   |
|Equestrian           |237  |
|Shooting             |342  |
|Diving               |133  |
|Canoe Slalom         |78   |
|Rugby Sevens         |283  |
|Artistic Gymnastics  |187  |
|Cycling Mountain Bike|74   |
|3x3 Basketball       |62   |
|Cycling R

### Task 3

3. Прочитайте исходный файл "Athletes.csv".  

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.  
Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников, только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.  
Результат сохраните в формате parquet.


In [18]:
athletesDF.show(truncate=False)

+-----------------------+------------------------+-------------------+
|Name                   |NOC                     |Discipline         |
+-----------------------+------------------------+-------------------+
|AALERUD Katrine        |Norway                  |Cycling Road       |
|ABAD Nestor            |Spain                   |Artistic Gymnastics|
|ABAGNALE Giovanni      |Italy                   |Rowing             |
|ABALDE Alberto         |Spain                   |Basketball         |
|ABALDE Tamara          |Spain                   |Basketball         |
|ABALO Luc              |France                  |Handball           |
|ABAROA Cesar           |Chile                   |Rowing             |
|ABASS Abobakr          |Sudan                   |Swimming           |
|ABBASALI Hamideh       |Islamic Republic of Iran|Karate             |
|ABBASOV Islam          |Azerbaijan              |Wrestling          |
|ABBINGH Lois           |Netherlands             |Handball           |
|ABBOT

In [19]:
athletesDF.count()

11085

In [20]:
athletes_countDF.show(10, truncate=False)

+---------------------+-----+
|Discipline           |count|
+---------------------+-----+
|Tennis               |178  |
|Boxing               |270  |
|Marathon Swimming    |49   |
|Golf                 |115  |
|Rowing               |496  |
|Baseball/Softball    |220  |
|Judo                 |373  |
|Sailing              |336  |
|Swimming             |743  |
|Cycling BMX Freestyle|19   |
+---------------------+-----+
only showing top 10 rows



In [21]:
olympicDF.show(100, truncate=False)

+------+--------------------+------+
|row_id|discipline          |season|
+------+--------------------+------+
|1     |skateboarding       |summer|
|2     |figure skating      |winter|
|3     |swimming            |summer|
|4     |ski jumping         |winter|
|5     |surfing             |summer|
|6     |snowboard           |winter|
|7     |golf                |summer|
|8     |ski mountaineering  |winter|
|9     |athletics           |summer|
|10    |biathlon            |winter|
|11    |badminton           |summer|
|12    |bobsleigh           |winter|
|13    |boxing              |summer|
|14    |cross-country skiing|winter|
|15    |triathlon           |summer|
|16    |speed skating       |winter|
+------+--------------------+------+



In [22]:
olympicDF.count()

16

In [23]:
athletes_count_joinedDF = olympicDF.join(athletes_countDF, olympicDF.discipline==lower(athletes_countDF.Discipline), how='left').select(olympicDF.discipline, 'count')

In [24]:
athletes_count_joinedDF.show()

+--------------------+-----+
|          discipline|count|
+--------------------+-----+
|       skateboarding|   77|
|      figure skating| null|
|            swimming|  743|
|         ski jumping| null|
|             surfing|   37|
|           snowboard| null|
|                golf|  115|
|  ski mountaineering| null|
|           athletics| 2068|
|            biathlon| null|
|           badminton|  164|
|           bobsleigh| null|
|              boxing|  270|
|cross-country skiing| null|
|           triathlon|  106|
|       speed skating| null|
+--------------------+-----+



In [25]:
athletes_count_joinedDF.repartition(1).write.mode("overwrite").parquet('file:///opt/notebooks/athletes_count_joined.parquet') 